# Sensitivities of crenulate bay shorelines to wave climates 

_This notebook is a companion to your practical and is based on the [COVE](http://cove-model.github.io/Documentation/) one-line model._


[COVE](http://cove-model.github.io/Documentation/) is a vector-based one-line model that we will use in the practical to explore how the wave direction $\theta_o$ and spreading $\theta_{std}$ is influencing shorelines evolution in embayment beaches like the ones around Sydney.

:::{seealso}
Part of the practical will require you to read through the paper from Hurst et al. (2015) available [here](https://agupubs.onlinelibrary.wiley.com/doi/full/10.1002/2015JF003704).
:::

## Running COVE

The [COVE](http://cove-model.github.io/Documentation/) model needs first to be installed on your system (own computer/remote sever). To do so you will have to run the following cell.

:::{note}
The `!` in `IPython Notebook` is a command `magic` which enable you to run terminal command directly from the code cell interface.
:::

In [2]:
!cd COVE/driver_files; make -f spiral_bay_make.make; mv cove ../../

It will create an executable named `cove` that will be used to run our different experiments. 

To run the model, you will use the following command (don't forget to put `!` before calling the executable).

In [18]:
!./cove

Program needs 6 input arguments:	 
	- MeanWavePeriod
	- StDWavePeriod
	- MeanWaveHeight
	- StDWaveHeight
	- MeanWaveDirection
	- StDWaveDirection


As shown in previous cell output the model requires 6 arguments. 

### Set the wave direction and spreading

We will give the following values for each of them: 
- MeanWavePeriod $T_{mean}=6s$ 
- StDWavePeriod $T_{std}=1s$ 
- MeanWaveHeight $H_{mean}=1.m$ 
- StDWaveHeight $H_{std}=0.1m$
- TimeStep $dt=0.2$ days
- EndTime $50$ years

And for this first run we will use the following value for the wave direction and spreading:
- MeanWaveDirection $\theta_{mean}$=$25^o$
- StDWaveDirection $\theta_{std}$=$10^o$

In [1]:
# Define wave climate params
Tmean = 6
Tstd = 1
Hmean = 1.0
Hstd = 0.1
Theta = 25
ThetaStd = 10
dt = 0.2
end = 30

In [2]:
!./cove $Tmean $Tstd $Hmean $Hstd  $Theta $ThetaStd $dt $end

zsh:1: permission denied: ./cove



## Plotting shoreline evolution

The model results are been written on disk in a file called `Spiral_6_1_1._0.1_25_10.xy`. As you can see the output file is named based on the input parameters given for the model run.

First we need to load the usually libraries (**numpy** and **matplotlib**).

In [3]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Customise figure style 
from matplotlib import rc
rc('font',size=13)
rc('ytick.major',pad=5)
rc('xtick.major',pad=5)
padding = 5

%config InlineBackend.figure_format = 'svg'

### Read the output file

We will now open the file and read the content:

In [ ]:
# Open coastline evolution file
filename = "Spiral_"+str(Tmean)+"_"+str(Tstd)+"_"+str(Hmean)+"_"+str(Hstd)+"_"+str(Theta)+"_"+str(ThetaStd)+".xy"

# Read the header info and get the lines
f = open(filename,"r")
Lines = f.readlines()
NoLines = len(Lines)-1
Header = Lines[0]

### Define output time to visualise

The outputs are created at specific time intervals (defined in years) and we will first specified the times we want to visualise:

In [4]:
# Times we want to extract from the output files
TimeIntervals = [0.2,0.5,1,2,4,8,16,20,30]

Time = np.zeros(NoLines-1)
for i in range(0,NoLines-1,2):
    Time[i] = float(Lines[i+1].strip().split(" ")[0])

NameError: name 'NoLines' is not defined

### Plot the results

We will now plot the result the wvolution of the given bay morphology toward an equilibrium morphology between the two fixed headlands (defined as the thick black lines). 

The plot will show the wave conditions defined by θmean and θstd as a wave rose. 

:::{note}
In the plot, model evolution starts from a straight coastline and the boundary conditions allow sediment to exit the model domain at either boundary by alongshore transport but sediment is not permitted to enter the bay from outside the domain.
:::

In [ ]:
# Define outputfile name based on theta nd thetastd values 
# (this file will be save on disk and could be used in your report)
outputfile = "mean"+ str(Theta) + "std" + str(ThetaStd) + ".png"

Let's make the plotting now. Each red dashed lines corresponds to a specific time interval defined by the `TimeIntervals` parameter above:

In [5]:
# Setup the figure space, axis and line to animate
fig = plt.figure(1, figsize=(6,6))
plt.subplots_adjust(0.2,0.15,0.9,0.85)


# Get the shoreline positions
X = np.array(Lines[1].strip().split(" ")[1:],dtype='float64')
Y = np.array(Lines[2].strip().split(" ")[1:],dtype='float64')
plt.plot(X,Y,'b.',ms=5,label="Initial Coastline")

plt.xlabel('X-coordinate (m)')
plt.ylabel('Y-coordinate (m)')

plt.xlim(-500,2000)
plt.ylim(-2100,0)

# Loop over the chosen time intervals
for TimeInt in TimeIntervals:
    Diff = np.abs(Time-TimeInt)
    Line = np.argmin(Diff)
    Xt = np.array(Lines[Line+1].strip().split(" ")[1:],dtype='float64')
    Yt = np.array(Lines[Line+2].strip().split(" ")[1:],dtype='float64')
    plt.plot(Xt,Yt,'r--',lw=1.5)

xmin = np.min(Xt)-500.
xmax = np.max(Xt)+500
ymin = np.min(Yt)-500.
ymax = np.max(Yt)+500

SeaX = np.append(Xt,[xmax,xmax,Xt[0]])
SeaY = np.append(Yt,[ymin,ymax,Yt[0]])
BeachX = np.append(Xt,[xmin,xmin,Xt[0]])
BeachY = np.append(Yt,[ymin,ymax,Yt[0]])
    
plt.fill(SeaX, SeaY, color=[0.7,0.9,1.0])
plt.fill(BeachX, BeachY, color=[1.0,0.9,0.6])

plt.plot(Xt,Yt,'r--',lw=1.5,label="Intermediate Coastlines")
plt.plot(Xt,Yt,'r-',lw=2,label="50 Year Coastline")
plt.plot(Xt,Yt,'ro',ms=6)

plt.plot(X[0:2],Y[0:2],'k-',lw=5)
plt.plot(X[-2:],Y[-2:],'k-',lw=5)

# Display legend
plt.rcParams.update({'legend.labelspacing':0.1}) 
plt.rcParams.update({'legend.numpoints':2}) 
plt.rcParams.update({'legend.frameon':False}) 
plt.rcParams.update({'legend.handlelength':1.5}) 
plt.legend(loc=3)
leg = plt.gca().get_legend()

# Set fontsize to small
ltext  = leg.get_texts()
plt.setp(ltext, fontsize=10) 

# Bin all wave data
Waves = np.random.normal(Theta,ThetaStd,10000)
widths = 15*np.pi/180.0
ax = plt.axes([0.65,0.55,0.25,0.25],polar=True)
hist = np.histogram(Waves*np.pi/180.0,bins=np.arange(0,361*np.pi/180.0,widths))
plt.bar(hist[1][:-1],hist[0],widths,color='white',edgecolor='k')
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_facecolor('none')
plt.axis('off')
plt.yticks([])
plt.xticks([])

plt.savefig(outputfile)
plt.show()

NameError: name 'Lines' is not defined

<Figure size 432x432 with 0 Axes>

## Clean run

To avoid storage of extensive data on the server delete the output file using the command `rm`. This is all right as you already have saved the model outputs as a `png` file!

In [ ]:
!rm Spiral_*.xy